## PlotDRFjs Notebook for ploting data from timeseries db Django Rest Framework (DRF) API, Using Javascript

This notebook uses javascript so it requires ijavascript (https://github.com/n-riesco/ijavascript) to be intalled. This will enable you to run a javascript kernal in the notebook. The first step to do this is to create a conda enviroment to install the software into. You can give this conda enviroment the name nodejs: 

    conda create --name nodejs python=3.9
    
After the conda enviroment has be created install activate it:

    conda activate tnodejs
    
Now install nodejs, and then jupyterlab, as follows: 

    conda install -c conda-forge nodejs

    conda install -c conda-forge jupyterlab

You can now install ijavascript using npm:

    npm install -g ijavascript
    
After ijavacript has been installed run the following command at you prompt:

    ijsinstall
    
To be able to see the graphs that are plotted an interface the can mimic a web browser is needed. There are many ways to do this. One of them is to use nteract (https://github.com/nteract/nteract). You can install nteract for jupyter using pip, as follows:

    pip install nteract_on_jupyter
    
This notebook uses danfojs (https://www.npmjs.com/package/danfojs-node), to process the data pulled from the timeseries db DRF API, and Plotly (https://github.com/n-riesco/ijavascript-plotly) to plot the graphs. Use npm to install these two packages, as follows:

    npm install danfojs-node
    
    npm install ijavascript-plotly
    
After these installations are complete the notebook is ready to run, using the following command:

    jupyter nteract
    
Notes: If you use Nteract to run this notebook, an use the "Run All Cells" command in the Cell menu, then the plotting will most likely fail, because the Promise of downloading the data has not been full filled. To avoid this problem, run each cell individually using the "Shit+Return" command.

In [1]:
// Get required modules
const dfd = require("danfojs-node");
const Plotly = require("ijavascript-plotly");

In [2]:
// This function takes the observation data pulled from timeseries db in json format, and converts it to danfojs dataframe for easier processing.
var getObs = function(observations) {
    let odt = [];
    let owl = [];
    let ods = [];

    for (var i = 0; i < observations.results.features.length; i++) {
        odt.push(observations.results.features[i].properties.time)
        owl.push(observations.results.features[i].properties.water_level)
        ods.push(observations.results.features[i].properties.data_source)
    };

    let obs_data = {
        datetime: odt,
        water_level: owl,
        data_source: ods,
    };

    return new dfd.DataFrame(obs_data);
};

In [3]:
// This function takes the namforecast data pulled from timeseries db in json format, and converts it to danfojs dataframe for easier processing.
var getForecast = function(forecast) {
    let fdt = [];
    let fwl = [];

    for (var i = 0; i < forecast.results.features.length; i++) {
        fdt.push(forecast.results.features[i].properties.time)
        fwl.push(forecast.results.features[i].properties.water_level)
    };

    let for_data = {
        datetime: fdt,
        water_level: fwl,
    };

    let fdfout = new dfd.DataFrame(for_data);
    fdfout.rename({ "water_level": "forecast" }, { inplace: true });
    return fdfout;
};

In [4]:
// This function takes the tidal predictions data pulled from timeseries db in json format, and converts it to danfojs dataframe for easier processing.
var getPredictions = function(predictions) {
    let pdt = [];
    let pwl = [];

    for (var i = 0; i < predictions.results.features.length; i++) {
        pdt.push(predictions.results.features[i].properties.time)
        pwl.push(predictions.results.features[i].properties.water_level)
    };

    let pre_data = {
        datetime: pdt,
        water_level: pwl,
    };

    let pdfout = new dfd.DataFrame(pre_data);
    pdfout.rename({ "water_level": "predictions" }, { inplace: true });
    return pdfout;
};

In [5]:
// This function takes a url, timemark, station name and gauge source name as input. It uses these variables to pull
// from the timeseries db in json format, and then converts this data to a form the can be used by plotly to plot a graph.
// There is an issue with the -99999 fill value used in the nowcast, and forecast data. The value is an integer, while the
// data values are float32. I need to talk to Jeff about changing the fill values to -99999.99.
var getData = async function(apiurl, timemark, station_name, gauge_source, nowcast_source, forecast_source) {
    // Start time count
    var start = performance.now();
    var end;
  
    // Create start date using timemark value.
    var tmpsdt = new Date(timemark);
    tmpsdt.setDate(tmpsdt.getDate() - 5);
    var startdt = tmpsdt.toISOString();
    
    // Create end date using timemark value.
    var tmpedt = new Date(timemark);
    tmpedt.setDate(tmpedt.getDate() + 3);
    tmpedt.setHours(tmpedt.getHours() + 9);
    var enddt = tmpedt.toISOString();
    
    // Create request urls fetch data from timeseries db.
    var requests = [
        fetch(apiurl+'gauge_station_source_data/?station_name='+station_name+'&data_source__in='+gauge_source+','+nowcast_source+',tidal_predictions&time__gt='+startdt+'&time__lt='+timemark+'&psize=5000&format=json').then((response) => {return response.json();}).then((data) => {observations = data;}),
        fetch(apiurl+'gauge_station_source_data/?station_name='+station_name+'&data_source='+forecast_source+'&timemark='+timemark+'&psize=5000&format=json').then((response) => {return response.json();}).then((data) => {forecast = data;}),
        fetch(apiurl+'gauge_station_source_data/?station_name='+station_name+'&data_source=tidal_predictions&time__gt='+timemark+'&time__lt='+enddt+'&psize=5000&format=json').then((response) => {return response.json();}).then((data) => {predictions = data;}),
  ];
  
    // Create results promise.
    const results = await Promise.allSettled(requests);
   
    // Defind variables.
    let output = '';
    let count = 0;
    
    // Check if promise has bee fulfilled/
    for (const result of results) {
       if (result.status === 'fulfilled') {
          output += `<p>Promise ${count+1 } fulfilled</p>`;
       } else {
          output += `<p>Promise ${count+1} rejected </p>`;
       }
       count++
    };
    
    // Convert observations data in json format to a danfojs dataframe.
    var odf = getObs(observations);
    
    // Extract nowcast data from observation dataframe and convert to form used for ploting.
    var dfnc = odf.loc({ rows: odf["data_source"].eq(nowcast_source) });
    dfnc.drop({ columns: ["data_source"], inplace: true });
    dfnc.rename({ "water_level": "nowcast" }, { inplace: true });
    
    // This section replaces the fill vaule -99999 with null, which gets interpreted by danfo.js as NaN values.
    // The value -99999 is an interger whild the actual data values are float32. Because of this replacing the 
    // -99999 value requires converting the nowcast colum to data type string to be able to replace the -99999
    // values with null values. The column nowcast is then converted back to float32. I need to talk to Jeff
    // about using -99999.99 as the fill value. I also need to talk to him about nowcast having fill values
    // for this station
    var dfncstr = dfnc.asType("nowcast", "string").replace("-99999", "null", { columns: ["nowcast"] });
    var dfnc = dfncstr.asType("nowcast", "float32")

    // Extract data values, from dataframe, for plotting.
    var nowtime = dfnc.datetime.values;
    var nowcast = dfnc.nowcast.values;
     
    // Extract tidal predictions data from observation dataframe and convert to form used for ploting.
    var dftp = odf.loc({ rows: odf["data_source"].eq('tidal_predictions') });
    dftp.drop({ columns: ["data_source"], inplace: true });
    dftp.rename({ "water_level": "predictions" }, { inplace: true });
    var pretime1 = dftp.datetime.values;
    var predictions1 = dftp.predictions.values;
    
    // Extract tidal gauge data from observation dataframe and convert to form used for ploting.
    var dftg = odf.loc({ rows: odf["data_source"].eq('tidal_gauge') });
    dftg.drop({ columns: ["data_source"], inplace: true });
    dftg.rename({ "water_level": "tidal_gauge" }, { inplace: true });
    var tidaltime = dftg.datetime.values;
    var tidal_gauge = dftg.tidal_gauge.values;

    // Convert namforecast data in json format to a danfojs dataframe, and convert to form used for ploting.
    var fdf = getForecast(forecast);
    
    // This section replaces the fill vaule -99999 with null, which gets interpreted by danfo.js as NaN values.
    // The value -99999 is an interger whild the actual data values are float32. Because of this replacing the 
    // -99999 value requires converting the forecast colum to data type string to be able to replace the -99999
    // values with null values. The column forecast is then converted back to float32. I need to talk to Jeff
    // about using -99999.99 as the fill value. I also need to talk to him about forecast having fill values
    // for this station
    var dfstr = fdf.asType("forecast", "string").replace("-99999", "null", { columns: ["forecast"] });
    var fdf = dfstr.asType("forecast", "float32")
    
    // Extract data values, from dataframe, for plotting.
    var foretime = fdf.datetime.values;
    var forecast = fdf.forecast.values;
    
    // Convert predictions data in json format to a danfojs dataframe, and convert to form used for ploting.
    var pdf = getPredictions(predictions);
    var pretime2 = pdf.datetime.values;
    var predictions2 = pdf.predictions.values;
     
    // Create add the above data to the dataout dictionary used for ploting data with plotly
    dataout = [
        {y: nowcast, x: nowtime, mode:"lines", name:"nowcast"},
        {y: tidal_gauge, x: tidaltime, mode:"lines", name:"tidal_gauge"},
        {y: predictions1, x: pretime1, mode:"lines", name:"predictions1"},
        {y: forecast, x: foretime, mode:"lines", name:"forecast"},
        {y: predictions2, x: pretime2, mode:"lines", name:"predictions2"}
    ];
    
    // End time count, caclulate time taken, and out put to conole.
    end = performance.now();
    console.log(`Execution time: ${end - start} ms`);
};

In [6]:
// This example pulls data from the timescale db on Sterling, at test-apsviz-timeseries.apps.renci.org, anb plots it. 
// Define input variables
var dataout;

// var apiurl = 'https://test-apsviz-timeseries.apps.renci.org/api/'
// timemark = '2023-02-02T07:00:00Z'
// run getData using the above inputs
getData('https://test-apsviz-timeseries.apps.renci.org/api/','2023-02-02T07:00:00Z','8651370','tidal_gauge','nowcast_ncsc_sab_v1.23','namforecast_ncsc_sab_v1.23');

Promise { <pending> }

(node:7884) ExperimentalWarning: The Fetch API is an experimental feature. This feature could change at any time
(Use `node --trace-warnings ...` to show where the warning was created)


Execution time: 1578.4290389977396 ms


In [7]:
// Plot results using plotly
var layout = {title: "Plot of data extracted from test database in eds namespace on Sterling"};
Plotly(dataout, layout);

In [8]:
// This example pulls data from the timescale db on the VM, at apsviz-timeseriesdb.edc.renci.org, and plots it. 
// Define input variables
var dataout;

// var apiurl = 'http://apsviz-timeseriesdb.edc.renci.org/api/';
// timemark = '2022-02-15T18:00:00Z'
// timemark = '2022-01-09T00:00:00Z'
// run getData using the above inputs
getData('http://apsviz-timeseriesdb.edc.renci.org/api/','2022-01-09T00:00:00Z','8651370','tidal_gauge','nowcast_hsofs','namforecast_hsofs');

Promise { <pending> }

Execution time: 4250.807074000128 ms


In [9]:
// Plot results using plotly
var layout = {title: "Plot of data extracted from test database on the apsviz-timeseriesdb.edc.renci.org VM"};
Plotly(dataout, layout);